In [19]:
#%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import warnings
warnings.simplefilter(action='ignore')
from sklearn.ensemble import RandomForestClassifier

In [20]:
test = pd.read_csv('test_label_enc.csv')
train = pd.read_csv('train_label_enc.csv')
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [21]:
train.columns

Index(['ID', 'Opportunity_ID', 'Cantidad_de_Items_Asociados', 'Region',
       'Territory', 'PricingDelivery_Terms_Quote_Appr',
       'PricingDelivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       ...
       'Source_7', 'Source_9', 'year_entrega', 'mes_entrega',
       'tiempo_plan_end_star', 'tiempo_entrega_por_items',
       'tiempo_entrega_max', 'tiempo_entrega_min', 'tiempo_entrega_mean',
       'tiempo_entrega_std'],
      dtype='object', length=2804)

In [22]:
train.PricingDelivery_Terms_Approved.value_counts()

1    9835
0    7009
Name: PricingDelivery_Terms_Approved, dtype: int64

In [23]:
# Aca saco las columnas que no quiero que las tome mi algoritmo.
features = list(set(train.columns) - set(['ID','Stage','Sales_Contract_No','contrato','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_converted_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date'\
                                          ,'Total_moneda_local','ASP_Currency','Last_Modified_By','Brand','Bureaucratic_Code',\
                                         'tiempo_creacion_ulti_mod','ASP_converted','Product_Name','Opportunity_Owner',\
                                          'Account_Owner','Product_Family','Account_Name','Estacion_entrega']))
#features

In [24]:
train_pos= train.loc[train['Stage']==1,:].sample(7315)
train_neg = train.loc[train['Stage']==0,:].sample(7315)
train=pd.concat([train_pos,train_neg]).drop_duplicates(keep=False)
train= shuffle(train)

In [25]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [26]:
clf = RandomForestClassifier()

In [27]:
#clf = RandomForestClassifier(n_estimators=300, criterion='gini',min_samples_split=2, max_features= 0.7,max_depth= 15,
#                               random_state=0, n_jobs=-1, verbose = 1)

In [28]:
clf.fit(x_train[features], y_train)

RandomForestClassifier()

In [29]:
scores = cross_val_score(clf, train[features], train['Stage'], cv=3,scoring= 'neg_log_loss')
scores

array([-0.23205198, -0.23470617, -0.22087711])

In [30]:
abs(scores.mean())

0.22921175601150956